In [180]:
%matplotlib notebook
import numpy as np
import pandas as pd
import datetime as dt
from matplotlib import pyplot as plt

from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

from ipywidgets import interact, interactive, fixed, interact_manual,\
Button, Layout, HBox, VBox, Dropdown, Select, Text,BoundedFloatText
from IPython.display import display, clear_output
import warnings
warnings.filterwarnings('ignore')

In [480]:
Path = 'C:/Users/wesle/QuickData_FI.csv'
df = pd.read_csv(Path,delimiter = ',',header = 0)
df = df.set_index(pd.DatetimeIndex(df.datetime))
ClimData = df.drop(df.columns[0],axis=1)
ClimData = ClimData[['AirTC_Avg','RH_Samp','air_pressure']]
# ClimData['air_pressure']/=100

In [487]:
class Calc_Flux:
    def __init__(self,Obs):
        self.Obs = Obs
        self.fig,self.ax = plt.subplots(nrows = 1,ncols = 1,figsize = (6,4))
        self.Key_1 = [key for key in Obs.keys()]
        self.k1 = 0
        self.Key_2 = [key for key in Obs[self.Key_1[0]].keys()]
        self.Results = {}
        for key_1 in self.Key_1:
            self.Results[key_1.split('.')[0]]={}
            for key_2 in self.Key_2:
                if key_2 != 'Clim':
                    self.Results[key_1.split('.')[0]][key_2] = np.nan
        self.k2 = 1
        self.k3 = 0
        self.Display()
        
    def Display(self):
        self.Depth = self.Obs[self.Key_1[self.k1]][self.Key_2[self.k2]]['Depth']
        if self.Depth == 'Missing':
            while self.Depth == 'Missing' and self.k3 == 0:
                self.Next()
        if self.k3 == 0:
            self.Data = self.Obs[self.Key_1[self.k1]][self.Key_2[self.k2]]['Obs']
            self.Data.reset_index(inplace=True)
            self.ax.plot(self.Data['co2'],label='_nolegend_',color='grey')
            self.ax.set_title(self.Key_1[self.k1]+ ' '+self.Key_2[self.k2])
            self.x1 = self.Data.index.min()
            self.x2 = self.Data.index.max()
            self.Rho(self.Obs[Key[0]]['Clim'])
            interact(self.SlideLeft,x=(0,self.Data.index.max()))
            interact(self.SlideRight,x=(0,self.Data.index.max()))
            ly = Layout(width='150px', height='50px')
            self.Calc = Button(description='Calculate',layout=ly)
            self.Calc.on_click(self.Flux)
            self.nyxt = Button(description='Next',layout=ly)
            self.nyxt.on_click(self.Next)
            h = HBox([self.Calc,self.nyxt])
            display(h)
        else:
            self.Done = Button(description='Done!')
            display(self.Done)
            self.Fluxes['idx'] = self.Fluxes.index
            self.Fluxes['Position'] = self.Fluxes['idx'].str.get(0)
            self.Fluxes['ID'] = self.Fluxes['idx'].str.get(1)
            self.Fluxes = self.Fluxes.drop(['idx'],axis=1)
            self.Fluxes.to_csv('C:/FishIsland_2017/ChamberStationData/ChamberFluxes.csv')
        
    def Next(self,b=0):
        try:
            self.Obs[self.Key_1[self.k1]][self.Key_2[self.k2]]['Flux'] = self.co2_flux
        except:
#             self.Obs[self.Key_1[self.k1]][self.Key_2[self.k2]]['Flux'] = np.nan
            pass
            
        self.k2 += 1
        if self.k2 >= len(self.Key_2):
            self.k2 = 1
            self.k1 += 1
            try:
                self.Key_2 = [key for key in self.Obs[self.Key_1[self.k1]].keys()]
            except:
                self.Save()
                pass
        try:
            self.Depth = self.Obs[self.Key_1[self.k1]][self.Key_2[self.k2]]['Depth']
        except:
            pass
        if b != 0:
            try:
                clear_output(wait=True)
                self.fig,self.ax = plt.subplots(nrows = 1,ncols = 1,figsize = (6,4))
                self.Display()
            except:
                self.Save()
            
    def Rho(self,varz):
        T = varz['AirTC_Avg']+273.15
        RH = varz['RH_Samp']/100
        P = varz['air_pressure']
        p_sat = 6.1078*10**(7.5*T/(T+237.3))
        p_v = RH*p_sat
        p_d = P-p_v
        R_Dry = 8.314
        self.rho = (p_d*0.028964+p_v*0.018016)/(R_Dry*T)
        print(P,p_d,p_v)
        print(self.rho)
        
    def Flux(self,b):
        area = 0.0079 # area in m2
        molar_mass = 0.0289644 # kg/mol
        molar_density = self.rho / molar_mass
        extra_volume = float(self.Depth) * 0.01 * area
        volume = 0.0014 + extra_volume # volume in m3
        molar_mass = 0.0289644 # kg/mol
        molar_density = self.rho / molar_mass
        model = make_pipeline(PolynomialFeatures(1), Ridge(alpha = 0))
        X =self.Slice.index.values.reshape(self.Slice.index.shape[0],1)
        y = self.Slice['co2'].values.reshape(self.Slice.index.shape[0],1)
        model.fit(X,y)
        self.Coefs=(model.steps[1][1].coef_[0][1])
        try:
            self.slyce.remove()
        except:
            pass
        self.Slice['Modeled'] = model.predict(X)
        self.co2_flux = molar_density * volume * self.Coefs / area
        self.slyce, = self.ax.plot(self.Slice['Modeled'],color='red',
                                   label='Flux :'+str(np.round(self.co2_flux,2)))
        print(self.co2_flux,molar_density * volume ,molar_density * volume /area)
        
        self.Results[self.Key_1[self.k1].split('.')[0]][self.Key_2[self.k2]] = self.co2_flux
        plt.legend()
        
    def SlideLeft(self,x):
        self.Slice = self.Data[x:self.x2]
        try:
            self.udate.remove()
        except:
            pass
        self.udate, = self.ax.plot(self.Slice['co2'],color='green',label='_nolegend_',
                                  linewidth=4)
        self.x1 = x
        return(x)
    
    def SlideRight(self,x):
        self.Slice = self.Data[self.x1:x]
        try:
            self.udate.remove()
        except:
            pass
        self.udate, = self.ax.plot(self.Slice['co2'],color='green',label='_nolegend_',
                                  linewidth=4)
        self.x2 = x
        return(x)
    
    def Save(self):
        self.k3 = 1
        self.Fluxes = pd.DataFrame(data = self.Results)
        clear_output(wait=False)
    

# Data = D.copy()
# print(Mod.Obs['6_22.DAT'])

In [488]:
class PreProcessing_Module:
    def __init__(self,ClimData):
        self.ClimData = ClimData
        self.Files = ['6_22.DAT','7_10.DAT','8_21.DAT']
        self.Times = ['2017-06-22 17:00:00','2017-07-10 15:00:00','2017-08-21 17:00:00']
        self.Collars = ['1A','1B','1C','1D','2A','2B','2C','2D','3A','3B','3C','3D']
        self.Dir = 'C:/FishIsland_2017/ChamberStationData/'
        self.Display()
        self.Done = 0
        
    def Display(self):
        self.Obs = {}
        for file in self.Files:
            self.Obs[file]={}
        dd = interactive(self.file_path,Run_File=self.Files)
        display(dd)
        ly = Layout(width='150px', height='50px')
        Start = Button(description='Plot',layout=ly)
        Start.on_click(self.start)
        Assign = Button(description='Assign',layout=ly)
        Assign.on_click(self.Assign)
        Drop = Button(description='Drop',
           layout=ly)
        Drop.on_click(self.Drop)
        Save = Button(description='Save',
           layout=ly)
        Save.on_click(self.Save)
        Clear = Button(description='Clear',
           layout=ly)
        Clear.on_click(self.Clear)
        Calc = Button(description='Calculate Fluxes',
           layout=ly)
        Calc.on_click(self.Calc)
        a = HBox([Start,Assign,Drop])
        b = HBox([Save,Clear,Calc])
        v = VBox([a,b])
        display(v)
        self.i = 0
        self.ID = []
        self.IDDD = Text(
                    value=self.Collars[self.i],
                    placeholder='Type something',
                    description='Collar ID:',
                    disabled=False)
        self.Depth = BoundedFloatText(
                    value=5.0,
                    min=1,
                    max=10.0,
                    step=0.1,
                    description='Collar Depth:',
                    disabled=False)
        h = HBox([self.IDDD,self.Depth])
        display(h)
        plt.figure(figsize=(6,4))
        
    def file_path(self,Run_File):
        self.i = 0
#         self.DDUdate()
        self.Path = self.Dir+Run_File
        self.f_i = np.where(np.asanyarray(self.Files)==Run_File)[0][0]
        self.f = Run_File
        print(self.ClimData.loc[self.Times[self.f_i]])
        
    def DDUdate(self):
#         try:
        print(self.i)
        if self.i < len(self.Collars):
            self.IDDD.value=self.Collars[self.i]
#         else:
#             
    
    def start(self,b):
        self.i=0
        self.IDDD.value=self.Collars[self.i]
        Data = pd.read_csv(self.Path,delimiter = ',',header = None,na_values = -9999)
        if Data[3].min()<1000:
            Data[3]+=1000
        Data['co2'] = Data[8]
        Data['RH'] = Data[10]
        self.Data = Data[['co2','RH']]
        try:
            plt.gcf().clear()
        except:
            pass
        
        self.Data['co2_diff'] = self.Data['co2'].diff()
        self.Data.loc[self.Data['co2_diff']<-5,['co2']]=np.nan
        splits = pd.isnull(self.Data).any(1).nonzero()[0]
        self.Samples = np.split(self.Data,splits[1:],axis=0)
        plt.plot(self.Data['co2'])
        self.udate, = plt.plot(self.Samples[self.i]['co2'],linestyle = '--',color='red')
        
    def Assign(self,b):
        try:
            self.udate.remove()
        except:
            pass
        if self.i<len(self.Samples):
            if self.i > 0 and self.IDDD.value == self.ID[-1]:
                self.Samples[self.i-1]=self.Samples[self.i-1].append(self.Samples[self.i])
                self.Samples[self.i-1].ffill(inplace=True)
                self.Drop(1)
                self.DDUdate()
            else:
                self.i +=1
                self.ID.append(self.IDDD.value)
                self.Itt()
                try:
                    self.udate, = plt.plot(self.Samples[self.i]['co2'],linestyle = '--',color='red')
                    self.DDUdate()
                except:
                    pass
        
        print(self.ID)
        
    def Drop(self,b):
        try:
            self.udate.remove()
        except:
            pass
        if self.i<len(self.Samples):
            del(self.Samples[self.i])
            self.Itt()
            if self.i<len(self.Samples):
                self.udate, = plt.plot(self.Samples[self.i]['co2'],linestyle = '--',color='red')
            
    def Save(self,b):
        i = 0
        self.Obs[self.f]={}
        self.Obs[self.f]['Clim'] = {}
        for col in self.ClimData:
            self.Obs[self.f]['Clim'][col]=self.ClimData[col].loc[self.Times[self.f_i]]
        for c in self.Collars:
            self.Obs[self.f][c] = {}
            self.Obs[self.f][c]['Obs']='Missing'
            self.Obs[self.f][c]['Depth']='Missing'
        for samp in self.Samples:
            print(i,self.ID[i])
            self.Obs[self.f][self.ID[i]]['Obs']=samp
            self.Obs[self.f][self.ID[i]]['Depth']=self.Depth.value
            plt.plot(samp['co2'],color='green')
            i +=1    
        self.ID = []                
        
    def Itt(self):
        if self.i<len(self.Samples)-1:
            while((np.isfinite(self.Samples[self.i]['co2'].mean())==False)and(self.i<len(self.Samples)-1)):
                del(self.Samples[self.i])
    
    def Clear(self,b):
        clear_output(wait=True)
        self.Display()
                
    def Calc(self,b):
#         print('kitty')
#         print(self.Obs)
        clear_output(wait=True)
        self.CF = Calc_Flux(self.Obs)
#         self.Results = CF.Fluxes
#         self.Done =1
#         print(self.Results)

Mod = PreProcessing_Module(ClimData)
# print(Mod.Flux)

In [505]:
# for key in Mod.Obs.keys():
#     for ky in Mod.Obs[key]:
#         if ky != 'Clim':
#             try:
#                 print(Mod.Obs[key][ky]['Obs'].loc[0])
#             except:
#                 print('Problem!!!!!!!!!!!!!!!', key,ky)
#                 print()
#                 pass
#     print(key,Mod.Obs[key].keys())
F = Mod.CF.Fluxes
print(F)

F['idx'] = F.index
F['Position'] = F['idx'].str.get(0)
F['ID'] = F['idx'].str.get(1)
F = F.drop(['idx'],axis=1)
F.to_csv('C:/FishIsland_2017/ChamberStationData/ChamberFluxes.csv')
Center6 = F.loc[F['Position']=='1']['6_22']
print(Center6)
Center7 = F.loc[F['Position']=='1']['7_10']
print(Center7)
Center8 = F.loc[F['Position']=='1']['8_21']
print(Center8)


Rim6 = F.loc[F['Position']=='2']['6_22']
Rim7 = F.loc[F['Position']=='2']['7_10']
Rim8 = F.loc[F['Position']=='2']['8_21']
print(Center8)


print(F.groupby(F['Position']).mean())
print(F.groupby(F['Position']).std())
# print(F.groupby(F['']))

        6_22       7_10      8_21 idx Position ID
1A  1.741065   1.480309  0.889123  1A        1  A
1B  1.766820   1.369242  1.037903  1B        1  B
1C  2.041939   1.049141  1.536464  1C        1  C
1D       NaN   1.281990  0.975178  1D        1  D
2A       NaN   3.784771  2.544567  2A        2  A
2B       NaN   2.807494  2.315192  2B        2  B
2C       NaN  15.589741  4.619025  2C        2  C
2D       NaN   2.147647  2.832546  2D        2  D
3A       NaN   4.499922  1.428422  3A        3  A
3B       NaN   1.547631  1.125923  3B        3  B
3C       NaN   1.304946  0.705230  3C        3  C
3D       NaN   1.590451  0.818033  3D        3  D
1A    1.741065
1B    1.766820
1C    2.041939
1D         NaN
Name: 6_22, dtype: float64
1A    1.480309
1B    1.369242
1C    1.049141
1D    1.281990
Name: 7_10, dtype: float64
1A    0.889123
1B    1.037903
1C    1.536464
1D    0.975178
Name: 8_21, dtype: float64
1A    0.889123
1B    1.037903
1C    1.536464
1D    0.975178
Name: 8_21, dtype: float64
  

In [508]:
import plotly.plotly as py
import plotly.graph_objs as go

import numpy as np

y0 = Center6# np.random.randn(50)-1
y01= Rim6
y1 = Center7#np.random.randn(50)+1
y11 = Rim7
y2 = Center8
y21 = Rim8

Center_6_22 = go.Box(
    y=y0,name = '6/22_a'
)
Center_7_10 = go.Box(
    y=y1,name = '7/10_a'
)
Center_8_21 = go.Box(
    y=y2,name = '8/21_a'
)

Rim_6_22 = go.Box(
    y=y01,name = '6/22_b'
)
Rim_7_10 = go.Box(
    y=y11,name = '7/10_b'
)
Rim_8_21 = go.Box(
    y=y21,name = '8/21_b'
)
data = [Center_6_22, Rim_6_22, Center_7_10, Rim_7_10, Center_8_21, Rim_8_21]
py.iplot(data)

In [524]:
X = ['6/22','7/10','8/22']
M = F.groupby(F['Position']).mean().T
S = F.groupby(F['Position']).std().T
Y = M['1']
y_s = S['1']
# M = F.groupby(F['Position']).mean().T
# S = F.groupby(F['Position']).std().T
Y2 = M['2']
y_s2 = S['2']

Y3 = M['3']
y_s3 = S['3']

In [527]:
import plotly.plotly as py
import plotly.graph_objs as go

data = [go.Scatter(
        x=X,
        y=Y2,
        error_y=dict(
            type='data',
            array=y_s2,
            visible=True
        ),
    name = 'Rim'
    ),
    go.Scatter(
        x=X,
        y=Y3,
        error_y=dict(
            type='data',
            array=y_s3,
            visible=True
        ),
        name='Transition'
    ),
    go.Scatter(
        x=X,
        y=Y,
        error_y=dict(
            type='data',
            array=y_s,
            visible=True
        ),
        name='Center'
    )
]

py.iplot(data, filename='basic-error-bar')